In [1]:
from google.colab import drive
drive.mount('/drive')
f = open('/drive/My Drive/Colab Notebooks/Mon_Homework1/clothing_shoes_jewelry.txt', 'rU') 

Mounted at /drive


<ipython-input-1-999ca5fb8463>:3: DeprecationWarning: 'U' mode is deprecated
  f = open('/drive/My Drive/Colab Notebooks/Mon_Homework1/clothing_shoes_jewelry.txt', 'rU')


In [2]:
#extracting data from the text file
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

my_dictionary = {}

reviewText=[]
value2=[]
for line in f:

  if line.startswith("reviewText:"):
    z=line.strip()
    z=z.split("reviewText:")
    reviewText.append(z[0])
    value2.append(z[1])
    z.clear()

  else:
    continue

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
#creating a function to build a dictionary from lists
def add_values_in_dict(sample_dict, key, list_of_values):

    if key not in sample_dict:
        sample_dict[key] = list()
    sample_dict[key].extend(list_of_values)
    return sample_dict

In [4]:
my_dictionary = add_values_in_dict(my_dictionary, 'reviewText', value2)


In [5]:
#converting dictionary to a dataframe
import pandas as pd
data = pd.DataFrame.from_dict(my_dictionary)
data=data[:][0:2000]
data.head()

,reviewText
0,This is a great tutu and at a really great pri...
1,I bought this for my 4 yr old daughter for dan...
2,What can I say... my daughters have it in oran...
3,"We bought several tutus at once, and they are ..."
4,Thank you Halo Heaven great product for Little...


### Data Pre-processing

In [6]:
#removing digits from textual data
def remove_digits(text):
  ans = ''.join([i for i in text if not i.isdigit()])
  return ans

data['reviewText'] =data['reviewText'].apply(remove_digits)
data1=data
data.head()

,reviewText
0,This is a great tutu and at a really great pri...
1,I bought this for my yr old daughter for danc...
2,What can I say... my daughters have it in oran...
3,"We bought several tutus at once, and they are ..."
4,Thank you Halo Heaven great product for Little...


In [7]:
#function for lemmatization in order to reduce anomalies that words or texts might have
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

data['reviewText'] = data['reviewText'].apply(lambda text: lemmatize_words(text))

data.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


,reviewText
0,This is a great tutu and at a really great pri...
1,I bought this for my yr old daughter for dance...
2,What can I say... my daughter have it in orang...
3,"We bought several tutu at once, and they are g..."
4,Thank you Halo Heaven great product for Little...


In [8]:
#extracting sentences that have either exclamtion marks or adjectives or all Upper case words or any combination of these as they might be informative in sentiment analysis.
#I used regex to extract these sentences 
import re
def combine(x):
  x =x.split(".")
  patternForAdj=re.compile(r"\b\w+able\b|\b\w+ic\b|\b\w+less\b|\b\w+ful\b|\b\w+ish\b|\b\w+ous\b|\b\w+some\b|\b\w+iant\b|\b\w+ient\b|\b\w+ible\b|\b\w+ive\b|\b\w+al\b")
  patternForEM=re.compile(r"[!]+")
  patternForCapW=re.compile(r"[A-Z][a-z]+")

  s=[y for y in x if patternForCapW.search(y) or patternForEM.search(y) or patternForAdj.search(y) ]
  s=".".join(s)
  return s

sent1= "this is a GOOD book."
s = combine(sent1)
print(s)

In [9]:
data['extractedSent'] = data['reviewText'].apply(combine)
data.head()


,reviewText,extractedSent
0,This is a great tutu and at a really great pri...,This is a great tutu and at a really great pri...
1,I bought this for my yr old daughter for dance...,I bought this for my yr old daughter for dance...
2,What can I say... my daughter have it in orang...,What can I say. It is a very good way for exal...
3,"We bought several tutu at once, and they are g...","We bought several tutu at once, and they are g..."
4,Thank you Halo Heaven great product for Little...,Thank you Halo Heaven great product for Little...


In [10]:
#calculating length of extracted sentences using len() function.
data['LengthOfSent']=data['extractedSent'].apply(len)
data.head()

,reviewText,extractedSent,LengthOfSent
0,This is a great tutu and at a really great pri...,This is a great tutu and at a really great pri...,167
1,I bought this for my yr old daughter for dance...,I bought this for my yr old daughter for dance...,187
2,What can I say... my daughter have it in orang...,What can I say. It is a very good way for exal...,150
3,"We bought several tutu at once, and they are g...","We bought several tutu at once, and they are g...",396
4,Thank you Halo Heaven great product for Little...,Thank you Halo Heaven great product for Little...,361


In [11]:
#Function for extracting adjectives using POS tag to get words that have adjective as a tag
import nltk
nltk.download('averaged_perceptron_tagger')
from textblob import TextBlob
def findAdj(x):
  

  sentence = TextBlob(x)
  sentence= [word for (word,tag) in sentence.tags if tag == "JJ"]
  sentence =list(set(sentence))
  if len(sentence) == 0:
    sentence = 0

  return sentence



[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [12]:
data['AdjectiveList']=data['extractedSent'].apply(findAdj)
data.head()

,reviewText,extractedSent,LengthOfSent,AdjectiveList
0,This is a great tutu and at a really great pri...,This is a great tutu and at a really great pri...,167,"[cheap, great, affordable, glad]"
1,I bought this for my yr old daughter for dance...,I bought this for my yr old daughter for dance...,187,"[old, adorable, first, good]"
2,What can I say... my daughter have it in orang...,What can I say. It is a very good way for exal...,150,"[comfortable, good, little, great, easy]"
3,"We bought several tutu at once, and they are g...","We bought several tutu at once, and they are g...",396,"[difficult, several, old, well-made, high, un-..."
4,Thank you Halo Heaven great product for Little...,Thank you Halo Heaven great product for Little...,361,"[long, purple, Little, great, Real]"


In [13]:
#function for extracting capital words using regex
def findCapitalWords(x):
  patternForCapW=re.compile(r"\b[A-Z][A-Z]+\b")
  CapWordList=patternForCapW.findall(x)
  if len(CapWordList) == 0 :
    CapWordList=0
  return CapWordList


In [14]:
data['CapitalWordsList']=data['extractedSent'].apply(findCapitalWords)
data.head()

,reviewText,extractedSent,LengthOfSent,AdjectiveList,CapitalWordsList
0,This is a great tutu and at a really great pri...,This is a great tutu and at a really great pri...,167,"[cheap, great, affordable, glad]",0
1,I bought this for my yr old daughter for dance...,I bought this for my yr old daughter for dance...,187,"[old, adorable, first, good]",0
2,What can I say... my daughter have it in orang...,What can I say. It is a very good way for exal...,150,"[comfortable, good, little, great, easy]",0
3,"We bought several tutu at once, and they are g...","We bought several tutu at once, and they are g...",396,"[difficult, several, old, well-made, high, un-...",0
4,Thank you Halo Heaven great product for Little...,Thank you Halo Heaven great product for Little...,361,"[long, purple, Little, great, Real]",[FL]


In [15]:
#getting list of words from extracted sentences that are recognized as Adjectives Or Verbs 
def filteringAV(x):
  sentence = TextBlob(x)
  sent = [word for (word,tag) in sentence.tags if tag == "JJ" or tag == "VB"]
  sent=list(set(sent))
  return sent

sent1 = "this is a great book. You must read it"
sent2 = filteringAV(sent1)
print(sent2)

['great', 'read']


In [16]:
data['AdjandVerbs']=data['extractedSent'].apply(filteringAV)
data.head()

,reviewText,extractedSent,LengthOfSent,AdjectiveList,CapitalWordsList,AdjandVerbs
0,This is a great tutu and at a really great pri...,This is a great tutu and at a really great pri...,167,"[cheap, great, affordable, glad]",0,"[affordable, cheap, great, look, glad]"
1,I bought this for my yr old daughter for dance...,I bought this for my yr old daughter for dance...,187,"[old, adorable, first, good]",0,"[old, adorable, first, good]"
2,What can I say... my daughter have it in orang...,What can I say. It is a very good way for exal...,150,"[comfortable, good, little, great, easy]",0,"[wear, comfortable, good, little, great, look,..."
3,"We bought several tutu at once, and they are g...","We bought several tutu at once, and they are g...",396,"[difficult, several, old, well-made, high, un-...",0,"[play, grow, difficult, several, old, well-mad..."
4,Thank you Halo Heaven great product for Little...,Thank you Halo Heaven great product for Little...,361,"[long, purple, Little, great, Real]",[FL],"[cute, long, pink, purple, Little, great, Real..."


In [17]:
#function to get bigrams that have adjectives or verbs in it
def generateBigrams(x,y):
  nltk_tokens = nltk.word_tokenize(x)  	
  bigrams=list(nltk.bigrams(nltk_tokens))
  Output = []
  for i in bigrams:
    for a in y:
      if a in i:
        Output.append(i)
  Output = [" ".join(w) for w in Output]
  return Output


In [18]:
data['BigramsList']= data.apply(lambda row: generateBigrams(row['extractedSent'], row['AdjandVerbs']), axis=1)
data.head()

,reviewText,extractedSent,LengthOfSent,AdjectiveList,CapitalWordsList,AdjandVerbs,BigramsList
0,This is a great tutu and at a really great pri...,This is a great tutu and at a really great pri...,167,"[cheap, great, affordable, glad]",0,"[affordable, cheap, great, look, glad]","[a great, great tutu, really great, great pric..."
1,I bought this for my yr old daughter for dance...,I bought this for my yr old daughter for dance...,187,"[old, adorable, first, good]",0,"[old, adorable, first, good]","[yr old, old daughter, the first, first time, ..."
2,What can I say... my daughter have it in orang...,What can I say. It is a very good way for exal...,150,"[comfortable, good, little, great, easy]",0,"[wear, comfortable, good, little, great, look,...","[very good, good way, : great, great colors, ,..."
3,"We bought several tutu at once, and they are g...","We bought several tutu at once, and they are g...",396,"[difficult, several, old, well-made, high, un-...",0,"[play, grow, difficult, several, old, well-mad...","[bought several, several tutu, got high, high ..."
4,Thank you Halo Heaven great product for Little...,Thank you Halo Heaven great product for Little...,361,"[long, purple, Little, great, Real]",[FL],"[cute, long, pink, purple, Little, great, Real...","[Heaven great, great product, for Little, Litt..."


In [19]:
#function for checking if extracted sentence has exclamation or not if it has then you add 1 if not then 0.
def findExclamation(x):
  patternForEM=re.compile(r"[!]+")
  if patternForEM.search(x):
    exc=1
  else:
    exc=0
  return exc



In [20]:
data['ExclamationMark']=data['extractedSent'].apply(findExclamation)
data.head()

,reviewText,extractedSent,LengthOfSent,AdjectiveList,CapitalWordsList,AdjandVerbs,BigramsList,ExclamationMark
0,This is a great tutu and at a really great pri...,This is a great tutu and at a really great pri...,167,"[cheap, great, affordable, glad]",0,"[affordable, cheap, great, look, glad]","[a great, great tutu, really great, great pric...",0
1,I bought this for my yr old daughter for dance...,I bought this for my yr old daughter for dance...,187,"[old, adorable, first, good]",0,"[old, adorable, first, good]","[yr old, old daughter, the first, first time, ...",0
2,What can I say... my daughter have it in orang...,What can I say. It is a very good way for exal...,150,"[comfortable, good, little, great, easy]",0,"[wear, comfortable, good, little, great, look,...","[very good, good way, : great, great colors, ,...",0
3,"We bought several tutu at once, and they are g...","We bought several tutu at once, and they are g...",396,"[difficult, several, old, well-made, high, un-...",0,"[play, grow, difficult, several, old, well-mad...","[bought several, several tutu, got high, high ...",0
4,Thank you Halo Heaven great product for Little...,Thank you Halo Heaven great product for Little...,361,"[long, purple, Little, great, Real]",[FL],"[cute, long, pink, purple, Little, great, Real...","[Heaven great, great product, for Little, Litt...",0


In [21]:
data['ExclamationMark'].value_counts()

0    1495
1     505
Name: ExclamationMark, dtype: int64

In [22]:
col = "AdjandVerbs"
data1 = data.loc[:, data.columns != col]
data1.head()

,reviewText,extractedSent,LengthOfSent,AdjectiveList,CapitalWordsList,BigramsList,ExclamationMark
0,This is a great tutu and at a really great pri...,This is a great tutu and at a really great pri...,167,"[cheap, great, affordable, glad]",0,"[a great, great tutu, really great, great pric...",0
1,I bought this for my yr old daughter for dance...,I bought this for my yr old daughter for dance...,187,"[old, adorable, first, good]",0,"[yr old, old daughter, the first, first time, ...",0
2,What can I say... my daughter have it in orang...,What can I say. It is a very good way for exal...,150,"[comfortable, good, little, great, easy]",0,"[very good, good way, : great, great colors, ,...",0
3,"We bought several tutu at once, and they are g...","We bought several tutu at once, and they are g...",396,"[difficult, several, old, well-made, high, un-...",0,"[bought several, several tutu, got high, high ...",0
4,Thank you Halo Heaven great product for Little...,Thank you Halo Heaven great product for Little...,361,"[long, purple, Little, great, Real]",[FL],"[Heaven great, great product, for Little, Litt...",0


## BUILDING A CLASSIFIER USING BOW + NEGATION
### I decided to go with combination of BOW + Negation as it gave better results
### I also tried using a combination of BOW + Subjectivty Lexicon but it didn't perform as good as BOW+ Negation it was giving accuracy around 75% whereas the current model has a better accuracy of 76.4%
### I didn't include the code for BOW + Subjectivity Lexicon because of computing issues.

In [23]:
import nltk
nltk.download('sentence_polarity')
from nltk.corpus import sentence_polarity
import random

# get the sentence corpus and look at some sentences
sentences = sentence_polarity.sents()
print(len(sentences))
print(sentence_polarity.categories())

# sentences are already tokenized, print the first four sentences
for sent in sentences[:4]:
    print(sent)

# look at the sentences by category to see how many positive and negative
pos_sents = sentence_polarity.sents(categories='pos')
print(len(pos_sents))
neg_sents = sentence_polarity.sents(categories='neg')
print(len(neg_sents))

[nltk_data] Downloading package sentence_polarity to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping corpora/sentence_polarity.zip.


10662
['neg', 'pos']
['simplistic', ',', 'silly', 'and', 'tedious', '.']
["it's", 'so', 'laddish', 'and', 'juvenile', ',', 'only', 'teenage', 'boys', 'could', 'possibly', 'find', 'it', 'funny', '.']
['exploitative', 'and', 'largely', 'devoid', 'of', 'the', 'depth', 'or', 'sophistication', 'that', 'would', 'make', 'watching', 'such', 'a', 'graphic', 'treatment', 'of', 'the', 'crimes', 'bearable', '.']
['[garbus]', 'discards', 'the', 'potential', 'for', 'pathological', 'study', ',', 'exhuming', 'instead', ',', 'the', 'skewed', 'melodrama', 'of', 'the', 'circumstantial', 'situation', '.']
5331
5331


In [24]:
documents = [(sent, cat) for cat in sentence_polarity.categories() 
	for sent in sentence_polarity.sents(categories=cat)]

print(documents[0])
print(documents[-1])

(['simplistic', ',', 'silly', 'and', 'tedious', '.'], 'neg')
(['provides', 'a', 'porthole', 'into', 'that', 'noble', ',', 'trembling', 'incoherence', 'that', 'defines', 'us', 'all', '.'], 'pos')


In [25]:
random.shuffle(documents)
#applying bow and then filtering on basis of 2000 topmost frequent words
all_words_list = [word for (sent,cat) in documents for word in sent]
all_words = nltk.FreqDist(all_words_list)

word_items = all_words.most_common(2000)
word_features = [word for (word,count) in word_items]
print(word_features[:50])

['.', 'the', ',', 'a', 'and', 'of', 'to', 'is', 'in', 'that', 'it', 'as', 'but', 'with', 'film', 'this', 'for', 'its', 'an', 'movie', "it's", 'be', 'on', 'you', 'not', 'by', 'about', 'more', 'one', 'like', 'has', 'are', 'at', 'from', 'than', '"', 'all', '--', 'his', 'have', 'so', 'if', 'or', 'story', 'i', 'too', 'just', 'who', 'into', 'what']


In [26]:
#function to check which words from the features are present in our documents in order to develop featureset
def document_features(document, word_features):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['V_{}'.format(word)] = (word in document_words)
    return features

featuresets = [(document_features(d, word_features), c) for (d, c) in documents]




In [27]:
# this list of negation words includes some "approximate negators" like hardly and rarely
negationwords = ['no', 'not', 'never', 'none', 'nowhere', 'nothing', 'noone', 'rather', 'hardly', 'scarcely', 'rarely', 'seldom', 'neither', 'nor']


In [28]:
#The approach involves sequentially examining each word in the document and incorporating its features into the feature set. However, if a word appears after a negation word, 
#its feature is altered to reflect negation. The feature set initially includes 2000 word features and 2000 "Not" word features, all initialized as false.
def NOT_features(document, word_features, negationwords):
    features = {}
    for word in word_features:
        features['V_{}'.format(word)] = False
        features['V_NOT{}'.format(word)] = False
    # go through document words in order
    for i in range(0, len(document)):
        word = document[i]
        if ((i + 1) < len(document)) and ((word in negationwords) or (word.endswith("n't"))):
            i += 1
            features['V_NOT{}'.format(document[i])] = (document[i] in word_features)
        else:
            features['V_{}'.format(word)] = (word in word_features)
    return features
# define the feature sets considering negation
NOT_featuresets = [(NOT_features(d, word_features, negationwords), c) for (d, c) in documents]


In [29]:
#splitting the data into traon and test and checkig accuracy
train_set, test_set = NOT_featuresets[1000:], NOT_featuresets[:1000]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.777

In [30]:
#checking for precision,recall and F score for positive and negative sentences individually
import nltk.metrics
import collections
refsets = collections.defaultdict(set)
testsets = collections.defaultdict(set)
 
for i, (feats, label) in enumerate(test_set):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
print( 'pos precision:', nltk.precision(refsets['pos'], testsets['pos']) )
print( 'pos recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'pos F-measure:', nltk.f_measure(refsets['pos'], testsets['pos']) )
print( 'neg precision:', nltk.precision(refsets['neg'], testsets['neg']) )
print( 'neg recall:', nltk.recall(refsets['neg'], testsets['neg']) )
print( 'neg F-measure:', nltk.f_measure(refsets['neg'], testsets['neg']) )

pos precision: 0.7686116700201208
pos recall: 0.7795918367346939
pos F-measure: 0.7740628166160081
neg precision: 0.7852882703777336
neg recall: 0.7745098039215687
neg F-measure: 0.7798617966436328


### Testing few statements

In [33]:
nltk.download('punkt')
text = 'I am officially done with @kanyewest'
texttokens = nltk.word_tokenize(text)
inputfeatureset = NOT_features(texttokens, word_features, negationwords)
classifier.classify(inputfeatureset)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


'neg'

In [35]:

text = 'My depression will not let me work out'
texttokens = nltk.word_tokenize(text)
inputfeatureset = NOT_features(texttokens, word_features, negationwords)
classifier.classify(inputfeatureset)

'neg'

In [36]:

text = 'Today is a nice, sunny day'
texttokens = nltk.word_tokenize(text)
inputfeatureset = NOT_features(texttokens, word_features, negationwords)
classifier.classify(inputfeatureset)

'pos'

In [37]:

text = 'I am proud of myself for achieving my goals.'
texttokens = nltk.word_tokenize(text)
inputfeatureset = NOT_features(texttokens, word_features, negationwords)
classifier.classify(inputfeatureset)

'pos'

### Analysis of Review Texts answering questions like how many sentences in a particluar review are positive/negative?What's the count of negative/positive sentences?

In [38]:
newData=pd.DataFrame(data,columns=['extractedSent', 'reviewText'])
newData.head()

,extractedSent,reviewText
0,This is a great tutu and at a really great pri...,This is a great tutu and at a really great pri...
1,I bought this for my yr old daughter for dance...,I bought this for my yr old daughter for dance...
2,What can I say. It is a very good way for exal...,What can I say... my daughter have it in orang...
3,"We bought several tutu at once, and they are g...","We bought several tutu at once, and they are g..."
4,Thank you Halo Heaven great product for Little...,Thank you Halo Heaven great product for Little...


In [39]:

def spliRt(text):
  x =text.split(".")
  return x

In [40]:
#classify function on new data for that you need to generate featureset in  the same way you generated featureset for train data.
def classify(text):
  x =text.split(".")

  s = [nltk.word_tokenize(i) for i in x]
  NOT_featuresets = [NOT_features(i, word_features, negationwords) for i in s]
  return NOT_featuresets

In [41]:
#getting tags for the sentences i.e. pos or neg
def do(x):
  tag=[classifier.classify(i) for i in x]
  return tag

In [42]:

newData['Tagged Sentences'] = newData['extractedSent'].apply(classify)
newData.head()

,extractedSent,reviewText,Tagged Sentences
0,This is a great tutu and at a really great pri...,This is a great tutu and at a really great pri...,"[{'V_.': False, 'V_NOT.': False, 'V_the': Fals..."
1,I bought this for my yr old daughter for dance...,I bought this for my yr old daughter for dance...,"[{'V_.': False, 'V_NOT.': False, 'V_the': True..."
2,What can I say. It is a very good way for exal...,What can I say... my daughter have it in orang...,"[{'V_.': False, 'V_NOT.': False, 'V_the': Fals..."
3,"We bought several tutu at once, and they are g...","We bought several tutu at once, and they are g...","[{'V_.': False, 'V_NOT.': False, 'V_the': Fals..."
4,Thank you Halo Heaven great product for Little...,Thank you Halo Heaven great product for Little...,"[{'V_.': False, 'V_NOT.': False, 'V_the': Fals..."


In [43]:
newData['Sentences'] = newData['Tagged Sentences'].apply(do)
newData.head()

,extractedSent,reviewText,Tagged Sentences,Sentences
0,This is a great tutu and at a really great pri...,This is a great tutu and at a really great pri...,"[{'V_.': False, 'V_NOT.': False, 'V_the': Fals...","[pos, pos, neg]"
1,I bought this for my yr old daughter for dance...,I bought this for my yr old daughter for dance...,"[{'V_.': False, 'V_NOT.': False, 'V_the': True...","[pos, neg]"
2,What can I say. It is a very good way for exal...,What can I say... my daughter have it in orang...,"[{'V_.': False, 'V_NOT.': False, 'V_the': Fals...","[neg, pos]"
3,"We bought several tutu at once, and they are g...","We bought several tutu at once, and they are g...","[{'V_.': False, 'V_NOT.': False, 'V_the': Fals...","[pos, pos, neg, pos, neg, pos, neg]"
4,Thank you Halo Heaven great product for Little...,Thank you Halo Heaven great product for Little...,"[{'V_.': False, 'V_NOT.': False, 'V_the': Fals...","[neg, neg, neg, pos, neg, neg, neg, neg, pos]"


In [44]:
newData['Split RT'] = newData['extractedSent'].apply(spliRt)
newData.head()

,extractedSent,reviewText,Tagged Sentences,Sentences,Split RT
0,This is a great tutu and at a really great pri...,This is a great tutu and at a really great pri...,"[{'V_.': False, 'V_NOT.': False, 'V_the': Fals...","[pos, pos, neg]",[This is a great tutu and at a really great pr...
1,I bought this for my yr old daughter for dance...,I bought this for my yr old daughter for dance...,"[{'V_.': False, 'V_NOT.': False, 'V_the': True...","[pos, neg]",[I bought this for my yr old daughter for danc...
2,What can I say. It is a very good way for exal...,What can I say... my daughter have it in orang...,"[{'V_.': False, 'V_NOT.': False, 'V_the': Fals...","[neg, pos]","[What can I say, It is a very good way for ex..."
3,"We bought several tutu at once, and they are g...","We bought several tutu at once, and they are g...","[{'V_.': False, 'V_NOT.': False, 'V_the': Fals...","[pos, pos, neg, pos, neg, pos, neg]","[We bought several tutu at once, and they are ..."
4,Thank you Halo Heaven great product for Little...,Thank you Halo Heaven great product for Little...,"[{'V_.': False, 'V_NOT.': False, 'V_the': Fals...","[neg, neg, neg, pos, neg, neg, neg, neg, pos]",[Thank you Halo Heaven great product for Littl...


In [45]:
#function to get number of positive sentences in extracted sentences
def PositiveCount(x):
  pos=0
  for i in x:
    if i == "pos":
      pos=pos+1
    else:
      continue
  return pos


In [46]:
newData['No of Positive Sentences'] = newData['Sentences'].apply(PositiveCount)
newData.head()

,extractedSent,reviewText,Tagged Sentences,Sentences,Split RT,No of Positive Sentences
0,This is a great tutu and at a really great pri...,This is a great tutu and at a really great pri...,"[{'V_.': False, 'V_NOT.': False, 'V_the': Fals...","[pos, pos, neg]",[This is a great tutu and at a really great pr...,2
1,I bought this for my yr old daughter for dance...,I bought this for my yr old daughter for dance...,"[{'V_.': False, 'V_NOT.': False, 'V_the': True...","[pos, neg]",[I bought this for my yr old daughter for danc...,1
2,What can I say. It is a very good way for exal...,What can I say... my daughter have it in orang...,"[{'V_.': False, 'V_NOT.': False, 'V_the': Fals...","[neg, pos]","[What can I say, It is a very good way for ex...",1
3,"We bought several tutu at once, and they are g...","We bought several tutu at once, and they are g...","[{'V_.': False, 'V_NOT.': False, 'V_the': Fals...","[pos, pos, neg, pos, neg, pos, neg]","[We bought several tutu at once, and they are ...",4
4,Thank you Halo Heaven great product for Little...,Thank you Halo Heaven great product for Little...,"[{'V_.': False, 'V_NOT.': False, 'V_the': Fals...","[neg, neg, neg, pos, neg, neg, neg, neg, pos]",[Thank you Halo Heaven great product for Littl...,2


In [47]:
#function to get number of negative sentences in extracted sentences
def NegativeCount(x):
  neg=0
  for i in x:
    if i =="neg":
      neg=neg+1
    else:
      continue
  return neg

In [48]:
newData['No of Negative Sentences']=newData['Sentences'].apply(NegativeCount)
newData.head()

,extractedSent,reviewText,Tagged Sentences,Sentences,Split RT,No of Positive Sentences,No of Negative Sentences
0,This is a great tutu and at a really great pri...,This is a great tutu and at a really great pri...,"[{'V_.': False, 'V_NOT.': False, 'V_the': Fals...","[pos, pos, neg]",[This is a great tutu and at a really great pr...,2,1
1,I bought this for my yr old daughter for dance...,I bought this for my yr old daughter for dance...,"[{'V_.': False, 'V_NOT.': False, 'V_the': True...","[pos, neg]",[I bought this for my yr old daughter for danc...,1,1
2,What can I say. It is a very good way for exal...,What can I say... my daughter have it in orang...,"[{'V_.': False, 'V_NOT.': False, 'V_the': Fals...","[neg, pos]","[What can I say, It is a very good way for ex...",1,1
3,"We bought several tutu at once, and they are g...","We bought several tutu at once, and they are g...","[{'V_.': False, 'V_NOT.': False, 'V_the': Fals...","[pos, pos, neg, pos, neg, pos, neg]","[We bought several tutu at once, and they are ...",4,3
4,Thank you Halo Heaven great product for Little...,Thank you Halo Heaven great product for Little...,"[{'V_.': False, 'V_NOT.': False, 'V_the': Fals...","[neg, neg, neg, pos, neg, neg, neg, neg, pos]",[Thank you Halo Heaven great product for Littl...,2,7


In [49]:
df2 = newData[['reviewText','No of Positive Sentences','No of Negative Sentences']]
df2.head()

,reviewText,No of Positive Sentences,No of Negative Sentences
0,This is a great tutu and at a really great pri...,2,1
1,I bought this for my yr old daughter for dance...,1,1
2,What can I say... my daughter have it in orang...,1,1
3,"We bought several tutu at once, and they are g...",4,3
4,Thank you Halo Heaven great product for Little...,2,7
